## Домашнее задание №4 (курс "Практикум по программированию на языке Python")

### Выполнил: <font color='red'>Никольский Владимир Андреевич, МГУ, ВМК</font>

### Тема: Web-сервер для обучения и использования ML-моделей

#### Преподаватели: Мурат Апишев (mel-lain@yandex.ru) и Роман Ищенко (roman.ischenko@gmail.com)

**Выдана**: 03.05.2023

**Дедлайн**: 17.05.2023

**Среда выполнения**: Jupyter Notebook (Python 3.7+)

#### Правила:

Результаты выполнения задания:

- архив со скриптами и файлами Dockerfile, который 1-2 команды позволяет развернуть сервер, решающий поставленные в задании задачи
- Jupyter Notebook, где __весь код__ из скриптов дублируется (1 ячейка - 1 скрипт) с комментарием, содержащим информацию о том, из какого файла взят код и что верхнеуровнево этот код делает

__Максимальное число баллов за задание - 25__.

Готовое задание отправляется на почту преподавателя.

Задание выполняется самостоятельно. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов. Если вы нашли в Интернете какой-то специфичный код, который собираетесь заимствовать, обязательно укажите это в задании - наверняка вы не единственный, кто найдёт и использует эту информацию.

Удалять фрагменты формулировок заданий запрещается.

### Постановка задачи:

**Серверная часть (15 баллов):**

- В данной работе нужно написать многозадачный веб-сервер для обучения и инференса ML моделей. На старте сервер получает на вход (через .env) конфиг, в котором должны быть указаны 3 параметра: путь к директории для сохранения моделей внутри контейнера сервера, число ядер, доступных для обучения и максимальное число моделей, которые могут быть одновременно загружены для инференса.


- Сервер должен реализовывать следующие методы:
    - `fit(X, y, config)` - обучить модель и сохранить на диск по указанным именем
    - `predict(y, config)` - предсказать с помощью обученной и загруженной модели по её имени
    - `load(config)` - загрузить обученную модель по её имени в режим инференса
    - `unload(config)` - выгрузить загруженную модель по её имени
    - `remove(config)` - удалить обученную модель с диска по её имени
    - `remove_all()` - удалить все обученные модели с диска


- Содержимое конфигов и форматы данных предлагается продумать и реализовать самостоятельно
- Сервер должен иметь счётчик активных процессов. Максимальное число активных процессов соответствует числу ядер, переданному в конфиге при старте сервиса. Каждое обучение модели запускается в отдельном процессе и до своего завершения потребляет этот процесс. Один процесс всегда остаётся для сервера, в нём же загружаются и работают на инференс обученные модели
- Сервер должен корректно обрабатывать все граничные случаи (запуск обучения без свободных ядер, запуск инфренса свыше лимита, запросы с несуществующими именами моделей, запросы с дублирующимися именами моделей)
- В реализации должны поддерживаться не менее трёх дискриминативных моделей (т.е. принимающих на вход объекты и метки при обучении и предсказывающих метки для новых объектов)
- Сервер должен быть реализован на FastAPI
- Проект разворачивается с помощью выбранной библиотеки управления виртуальными окружениями и технологии контейнеризации Docker

**Клиентская часть (10 баллов):**

- Клиентская часть должна демонстрировать работу с реализованным сервером с помощью библиотек requests и aiohttp. Она может быть реализована непосредственно в Jupyter Notebook, с описанием ожидаемого действия, или в отдельном(-ых) скрипте(-ах), с дублированием в Jupyter Notebook (тогда работоспособность в ноутбуке не требуется). Далее описываются отдельные функции:
- Код вызова последовательного вызова обучения как минимум двух (N) различных моделей с таким набором данных и параметрами, чтобы обучение одной модели длилось не менее 60 секунд.
- Код вызова асинхронного вызова обучения как минимум двух различных моделей с демонстрацией, что работа выполняется в два (в N) раза быстрее
- Асинхронный вызов нескольких предсказаний
- Код демонстрации остальных функций сервера (загрузка, выгрузка, удаление)
- Должны обрабатываться ошибки и исключения, возвращаемые сервером


In [40]:
import pandas as pd
import numpy as np

import datetime

import asyncio
import aiohttp
import requests as r

import time
from functools import wraps

from sklearn.ensemble import (
    GradientBoostingClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import warnings

warnings.filterwarnings('ignore')

# Эксперименты

Перед написанием серверной части задания проведем эксперименты на синтаксических данных. Заметим, что обучение модели МО является CPU bound задачей, поэтому будем использовать библиотеку __multiprocessing__, чтобы обойти GIL.

In [41]:
# Декоратор для замера времени выполнения функции

def time_of_function(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        start_time = datetime.datetime.now()
        res = function(*args, **kwargs)
        print(datetime.datetime.now() - start_time)
        return res
    return wrapped

Будем экспериментировать на методе __GradientBoostingClassifier__. Выбор обусловлен тем, что на сравнительно небольшом датасете (5000 записей для 10 признаков) время обучения одной модели составлет около минуты.

# Клиентская часть

In [42]:
X = np.random.randint(0, 100, size=(5000, 10))
y = np.random.randint(0, 100, size=(5000,))

X = X.tolist()
y = y.ravel().tolist()

Запустим приложение командой __make run_all_docker__ и приступим к демонстрации работы серверной части

In [43]:
URL = 'http://0.0.0.0:8000'  # URL приложения на локальной машине

In [52]:
# Проверим, что сервер запущен и обрабатывает запросы

health_check = r.get(url=f'{URL}/health')
assert health_check.ok

In [53]:
# Соберем тело запроса для обучения. 

req_1_body = {
    "X": X,
    "y": y,
    "sync": 'No',
    "config":{
        "file_name": "boosting_classifier_1",
        "model": "GradientBoostingClassifier",
        "params":{
            "max_depth": 2
        }
    }
}

req_2_body = {
    "X": X,
    "y": y,
    "sync": 'No',
    "config":{
        "file_name": "boosting_classifier_2",
        "model": "GradientBoostingClassifier",
        "params":{
            "max_depth": 2
        }
    }
}

In [55]:
import nest_asyncio
nest_asyncio.apply()

async def async_requests():
    async with aiohttp.ClientSession() as session:
        req_1 = await session.post(url=f'{URL}/fit', json=req_1_body)
        req_2 = await session.post(url=f'{URL}/fit', json=req_2_body)
        
        assert req_1.ok
        assert req_2.ok
        
        print(f'Запросы обработаны успешно')
        
loop = asyncio.get_event_loop()
loop.run_until_complete(async_requests())

Запросы обработаны успешно


In [49]:
# Соберем тело запроса для обучения. 

req_1_body = {
    "X": X,
    "y": y,
    "sync": 'Yes',
    "config":{
        "file_name": "boosting_classifier_1",
        "model": "GradientBoostingClassifier",
        "params":{
            "max_depth": 2
        }
    }
}

req_2_body = {
    "X": X,
    "y": y,
    "sync": 'Yes',
    "config":{
        "file_name": "boosting_classifier_2",
        "model": "GradientBoostingClassifier",
        "params":{
            "max_depth": 2
        }
    }
}

In [51]:
import nest_asyncio
nest_asyncio.apply()

async def sync_requests():
    async with aiohttp.ClientSession() as session:
        req_1 = await session.post(url=f'{URL}/fit', json=req_1_body)
        req_2 = await session.post(url=f'{URL}/fit', json=req_2_body)
        
        assert req_1.ok
        assert req_2.ok
        
        print(f'Запросы обработаны успешно')
        
loop = asyncio.get_event_loop()
loop.run_until_complete(sync_requests())

Запросы обработаны успешно
